# DriveNet Close Loop Evaluation

**Note: this notebook assumes you've already run the [training notebook](./drivenet_train.ipynb) and stored your model successfully.**

### What is close loop evaluation?
In close loop evaluation DriveNet is in **full control of the AV**. At each time step, we predict the future trajectory and then we move the AV int he first of the DriveNet's predictions. 

For this setting metrics are particularly challenging. In fact, we would like to penalise some of the drifting (e.g. going off road or in the opposite lane) while at the same time allow others (e.g. different speed profiles)

Internally, we use a substantial sets of different metrics to capture dangerous manoeuvres and behaviours. For the sake of simplicity, in this notebook we will be using a very simple proxy to detect if our model is driving in a sensible way, **collisions with other agents**.

Although simple, when used in densily crowded area this metric is surprisingly effective.

In [ ]:
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.geometry import transform_points, angular_distance, yaw_as_rotation33
from l5kit.visualization import TARGET_POINTS_COLOR, PREDICTED_POINTS_COLOR, draw_trajectory
from l5kit.drivenet.model import DriveNetModel
from l5kit.kinematic import AckermanPerturbation
from l5kit.random import GaussianRandomGenerator

import os

## Prepare Data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/tmp/l5kit_data"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./drivenet_config.yaml")

## Load The Model



In [ ]:
model_path = "/var/folders/7f/mb3llzfs40b86yf2km8fml2h0000gp/T/drivenet.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path).to(device)
model = model.eval()
torch.set_grad_enabled(False)

## Load the Evaluation Data
Differently from training and open loop evaluation, this setting is intrinsically sequential. As such, we won't be using any parallelisation offered by pytorch.

In [ ]:
# ===== INIT DATASET
eval_cfg = cfg["val_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
eval_zarr = ChunkedDataset(dm.require(eval_cfg["key"])).open()
eval_dataset = EgoDataset(cfg, eval_zarr, rasterizer)
print(eval_dataset)

# Unroll the first scene

in this cell we unroll the first scene of the dataset

TODO generic function for this

TODO add collisions metric

In [ ]:
# ==== EVAL LOOP
scene_dataset = eval_dataset.get_scene_dataset(0)

images = []

for frame_idx in tqdm(range(len(scene_dataset)//5)):
    data = scene_dataset[frame_idx]
    del data["host_id"]
    data_batch = default_collate([data])
    result = model(data_batch)
    predicted_positions = result["positions"].detach().cpu().numpy().squeeze()
    predicted_yaws = result["yaws"].detach().cpu().numpy().squeeze()
    
    ## store image for future plot
    im_ego = rasterizer.to_rgb(data["image"].transpose(1, 2, 0))    
    draw_trajectory(im_ego, transform_points(predicted_positions, data["raster_from_agent"]), PREDICTED_POINTS_COLOR)
    images.append(im_ego[::-1])    
    
    ## mutate the next frame
    pred_positions_m = predicted_positions + data["centroid"]
    pred_angles_rad = predicted_yaws + data["yaw"]
    
    frame_mutate_idx = frame_idx + 1
    if frame_mutate_idx < len(scene_dataset):
        scene_dataset.dataset.frames[frame_mutate_idx]["ego_translation"][:2] = pred_positions_m[0]
        scene_dataset.dataset.frames[frame_mutate_idx]["ego_rotation"] = yaw_as_rotation33(pred_angles_rad[0])

# Qualitative Evaluation: Visualise the Close Loop

We can visualise the frames we have stored in the previous cell. As we have mutated future positions, DriveNet is now in full control of the AV as it moves through the annotated scene.


In [ ]:
from IPython.display import display, clear_output
import PIL
 
for frame in images:
    clear_output(wait=True)
    display(PIL.Image.fromarray(frame))

## Is open loop evaluation enough?

Depending on how long you've trained your model, you may have seen different results in the above cell.

If you've trained it long enough, the predicted trajectory is likely well overlapped with the annotated one.

Conversely, a model trained not enough will surely show some bias or unfeasible trajectories.

In both cases, **this evaluation is not enough** to ensure your model will be able to actually drive on the road (that's where we all want to go in the end). If your model is not in controll of where to go, you can't really say it will work once the annotated trajectory won't be available.

What we really want to do is to have our model in full control of the AV future positions, in a setting called **close loop**. To this end, you can find a notebook fully dedicated to that TODO HERE